## Data Engineering

In [1]:
#Importing Packages
import sys
import os
sys.path.append("..")
import pandas as pd
from FasttextClassifier.FasttextClassifier import FasttextClassifier
from sklearn.model_selection import train_test_split
import random
import csv
from pathlib import Path


## Creating FastText training file from Reuters files

In [2]:
train = pd.read_csv('../Data/train/Reuters/r8-train-all-terms.txt', header=None, sep='\t')
train.columns = ['label', 'content']
test = pd.read_csv('../Data/test/Reuters/r8-test-all-terms.txt', header=None, sep='\t')
test.columns = ['label', 'content']
test.head()

,label,content
0,trade,asian exporters fear damage from u s japan rif...
1,grain,china daily says vermin eat pct grain stocks a...
2,ship,australian foreign ship ban ends but nsw ports...
3,acq,sumitomo bank aims at quick recovery from merg...
4,earn,amatil proposes two for five bonus share issue...


In [3]:
test['label'] = '__label__'+test['label']
train['label'] = '__label__'+train['label']
train

,label,content
0,__label__earn,champion products ch approves stock split cham...
1,__label__acq,computer terminal systems cpml completes sale ...
2,__label__earn,cobanco inc cbco year net shr cts vs dlrs net ...
3,__label__earn,am international inc am nd qtr jan oper shr lo...
4,__label__earn,brown forman inc bfd th qtr net shr one dlr vs...
...,...,...
5480,__label__earn,kelly oil and gas partners kly year dec shr ct...
5481,__label__money-fx,japan seeks to strengthen paris currency accor...
5482,__label__earn,tcw convertible securities cvt sets dividend t...
5483,__label__money-fx,south korean won fixed at month high the bank ...


In [4]:
#saving train file
train.to_csv('trainFT.txt', sep='\t', header=None, index=False)
test.to_csv('testFT.txt', sep='\t', header=None, index=False)

### Creating the model and testing

In [5]:
ft_model = FasttextClassifier(train_data='trainFT.txt')

In [6]:
ft_model.fasttext_test('testFT.txt')

(2189, 0.9693924166285975, 0.9693924166285975)

In [7]:
ft_model.label('I like apples.')

(('__label__acq',), array([0.99466139]))

## Creating FastText model from raw files

In this example, we will take a dataset of raw files, divided into labels defined by their path.

The dataset shall follow the format:  
<pre>
.  
└── Dataset  
    └── label1  
    |   └── label_text.txt  
    ├── label2  
    |   ├── firstpost.txt  
    |   ├── happy.txt     
    |   └── secondpost.txt  
    └── label3  
        ├── text0.txt      
        └── second.txt    
</pre>
  

In [9]:

RAW_DATA_PATH = 'raw_data/'
GENERATED_DATA_PATH = 'generated_files/'
MERGED_DATA_PATH ='merged/'
SHUFFLED_MERGED_DATA_PATH = 'shuffled_merged/'
DATA_FINAL_PATH = 'data_final/'


In [10]:
def read_all_texts(path_to_folders, outfile_path):
    init_model=FasttextClassifier()
    for folder in os.listdir(f'{path_to_folders}'):
        for filename in os.listdir(f'{path_to_folders}/{folder}'):
            with open(f'{path_to_folders}/{folder}/{filename}', encoding='utf-8') as f:
                init_model.gen_training(path= f'{path_to_folders}/{folder}', label= f'{folder}', outfile_path=f'{outfile_path}/{folder}')

### Merging files

In [ ]:
def merge_files(path_to_folder, outfile_path):
    for file in os.listdir(f'{path_to_folder}'):
            with open(f'{path_to_folder}/{file}', encoding='utf-8') as fp: 
                data = fp.read() 
                data += '\n'
                file_lst=[]
                for filename in os.listdir(path_to_folder):
                    with open(f'{path_to_folder}/{filename}', encoding='utf-8') as f:
                        file_content = pd.read_csv(f, sep='\t', header=None)
                        file_lst.append(file_content)
                        
                merged = pd.concat(file_lst)
                merged.to_csv(f'{outfile_path}/' + 'merged', header=False, index=False, sep='\t')

In [ ]:
merge_files(GENERATED_DATA_PATH, MERGED_DATA_PATH)

### Shuffle the data

In [ ]:
def shuffler(path_to_folder, outfile_path):
    for file in os.listdir(f'{path_to_folder}'):
            with open(f'{path_to_folder}/{file}', encoding='utf-8') as fp:
                df=pd.read_csv(fp, sep='\t', header=None)
                df = df.sample(frac=1).reset_index(drop=True)
    return df
    df.to_csv(f'{outfile_path}/' + 'shuffled_merged' + '.txt', header=False, index=False, sep='\t')

In [ ]:
shuffled_merged=shuffler(MERGED_DATA_PATH, SHUFFLED_MERGED_DATA_PATH)

### Split train and test data

In [ ]:
X_train, X_test , y_train, y_test = train_test_split(shuffled_merged[1], shuffled_merged[0], test_size=0.33, random_state=42)

In [11]:
train=pd.DataFrame(X_train)
train.insert(0, '', y_train, True)
#saving train file
train.to_csv(DATA_FINAL_PATH + 'train_data.txt', sep='\t', header=None, index=False)

NameError: name 'X_train' is not defined

In [ ]:
test=pd.DataFrame(X_test)
test.insert(0, '', y_test, True)
#saving test file
test.to_csv(DATA_FINAL_PATH + 'test_data.txt', sep='\t', header=None, index=False)

### Load files and create model

In [ ]:
with open(DATA_FINAL_PATH + 'train_data.txt', encoding='utf-8') as train_txt:
    final_train_data=train_txt.read()

with open(DATA_FINAL_PATH + 'test_data.txt', encoding='utf-8') as test_txt:
    final_test_data=test_txt.read()

    
ft_model=FasttextClassifier(train_data=str(DATA_FINAL_PATH + 'train_data.txt'))
ft_model.fasttext_test(DATA_FINAL_PATH + 'test_data.txt')